## Data Wrangling: Join, Combine, and Reshape

### Hierarchical Indexing

*Hierarchical indexing* is an important feature of pandas that enables you to have multiple index *level* on an axis. It provides a way for you to work with higher dimensional data in a lower dimensional form.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                index = [['a','a','a','b','b','c','c','d','d'],
                        [1,2,3,1,3,1,2,3,1]])

data

a  1    1.789659
   2    0.427638
   3   -1.587755
b  1    0.274550
   3   -0.167158
c  1   -1.575036
   2    0.800977
d  3    0.190708
   1    2.250988
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 3),
            ('d', 1)],
           )

With a hierarchically indexed object, so-called *partial* indexing is possible, enable you to concisely select subsets of the data

In [4]:
data['b']

1    0.274550
3   -0.167158
dtype: float64

In [5]:
data['b':'c']

b  1    0.274550
   3   -0.167158
c  1   -1.575036
   2    0.800977
dtype: float64

In [6]:
data.loc[['b','d']]

b  1    0.274550
   3   -0.167158
d  3    0.190708
   1    2.250988
dtype: float64

In [7]:
data.loc[:, 2]

a    0.427638
c    0.800977
dtype: float64

In [8]:
data.unstack()

,1,2,3
a,1.789659,0.427638,-1.587755
b,0.274550,NaN,-0.167158
c,-1.575036,0.800977,NaN
d,2.250988,NaN,0.190708


In [9]:
data.unstack().stack()

a  1    1.789659
   2    0.427638
   3   -1.587755
b  1    0.274550
   3   -0.167158
c  1   -1.575036
   2    0.800977
d  1    2.250988
   3    0.190708
dtype: float64

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index = [['a','a','b','b'],[1,2,1,2]],
                    columns = [['Ohio','Ohio','NC'],['Green','Red','Green']])

In [11]:
frame

Ohio        NC
    Green Red Green
a 1     0   1     2
  2     3   4     5
b 1     6   7     8
  2     9  10    11

In [12]:
# The hierarchical levels can have names.
frame.index.names = ['key1','key2']

frame.columns.names = ['state','color']

frame

state      Ohio        NC
color     Green Red Green
key1 key2                
a    1        0   1     2
     2        3   4     5
b    1        6   7     8
     2        9  10    11

In [13]:
# With partial column indexing you can similarly select groups of columns
frame['NC']

color      Green
key1 key2       
a    1         2
     2         5
b    1         8
     2        11

In [14]:
# A MultiIndex can be created by itself and then reused; the columns in the preceding 
# DataFrame with level names
pd.MultiIndex.from_arrays([['Ohio','Ohio','NC'],
                          ['Green','Gold','Green']],
                         names = ['state','color'])

MultiIndex([('Ohio', 'Green'),
            ('Ohio',  'Gold'),
            (  'NC', 'Green')],
           names=['state', 'color'])

### Reordering and sorting levels

Need to rearrange the order of the levels on an axis or sort the data by the values in one specific level. The `swaplevel` takes two level numbers or names and returns a new object with the levels interchanged (but the data is otherwise unaltered)

In [15]:
frame.swaplevel('key1','key2')

state      Ohio        NC
color     Green Red Green
key2 key1                
1    a        0   1     2
2    a        3   4     5
1    b        6   7     8
2    b        9  10    11

`sort_index`, on the other hand, sorts the data using only the values in a single level. When swapping levels, it's not uncommon to also use `sort_index` so that the result is lexicographically sorted by the indicated level

In [16]:
frame.sort_index(level = 1)

state      Ohio        NC
color     Green Red Green
key1 key2                
a    1        0   1     2
b    1        6   7     8
a    2        3   4     5
b    2        9  10    11

In [17]:
frame.swaplevel(0,1).sort_index(level = 0)

state      Ohio        NC
color     Green Red Green
key2 key1                
1    a        0   1     2
     b        6   7     8
2    a        3   4     5
     b        9  10    11

**NOTE** Data selection performance is much better on hierarchically indexed objects if the index is lexicographically sorted starting with the outermost level.

---

### Summary statistics by level


In [18]:
frame.sum(level = 'key2')

state  Ohio        NC
color Green Red Green
key2                 
1         6   8    10
2        12  14    16

In [19]:
frame.sum(level = 'key1')

state  Ohio        NC
color Green Red Green
key1                 
a         3   5     7
b        15  17    19

### Indexing with a DataFrame's columns


In [20]:
frame = pd.DataFrame({'a':range(7),'b':range(7,0,-1),
                     'c':['one','one','one','two','two','two','two'],
                     'd':[0, 1, 3, 0, 1, 2, 3]})

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,3
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [21]:
# set_index function will create a new DataFrame using one or more of its columns as the index

frame2 = frame.set_index(['c','d'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    3  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [22]:
# By default the columns are removed from the DataFrame, though you can leave them in

frame.set_index(['c','d'], drop = False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    3  2  5  one  3
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [23]:
# reset_index does the opposite of set_index; the hierarchical index levels are moved into the columns

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,3,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## Combing and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:
* `pandas.merge` connects rows in DataFrames based on one or more keys. 
* `pandas.concat` concatenates or "stacks" together objects along an axis.
* The `combine_first` instance method enables splicing together overlapping data to fill in missing values in one object with values from another.

### Database-style DataFrame Joins

*Merge* or *join* operations combine datasets by linking rows using one or more *keys*. These operations are central to relational databases. The `merge` function in pandas is the main entry point for using these algorithms on data.

In [24]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],
                   'data2':range(3)})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [25]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [26]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [27]:
pd.merge(df1, df2, on = 'key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [28]:
# If the column names are different in each object, you can specify them separately

df3 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],
                   'data1':range(7)})
df4 = pd.DataFrame({'rkey':['a','b','d'],
                   'data2':range(3)})

pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [29]:
df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


You may notice that the '`c`' and '`d`' values and associated data are missing from the result. By default `merge` does an '`inner`' join; the keys in the result are the intersection, or the common set found in both tables. Other possible options are '`left`','`right`', and '`outer`'. The outer join takes the union of the keys, combining the effect of applying both left and right joins.

In [30]:
pd.merge(df1, df2, how = 'outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [31]:
# Many-to-many merges have well-defined, though not necessarily intuitive, behavior.

df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],
                   'data1':range(6)})

df2 = pd.DataFrame({'key':['a','b','a','d','b'],
                   'data2':range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [32]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,d,3
4,b,4


In [33]:
pd.merge(df1, df2, on = 'key', how = 'left')

,key,data1,data2
0,b,0,1.0
1,b,0,4.0
2,b,1,1.0
3,b,1,4.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


*Table 8-1 Different join types with how argument*

---

|**Option**|**Behavior**|
|:---:|:---|
|`inner`|Use only the key combinations observed in **both** tables|
|`left`|Use all key combinations found in the left table|
|`right`|Use all key combinations found in the right table|
|`outer`|Use all key combinations observed in **both** tables **together**|

In [34]:
# Many-to-many joins form the Cartesian product of the rows. 
# The join method only affects the distinct key values appering the result

pd.merge(df1, df2, how = 'inner')

,key,data1,data2
0,b,0,1
1,b,0,4
2,b,1,1
3,b,1,4
4,b,5,1
5,b,5,4
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [35]:
# To merge with multiple keys, pass a list of column names:

left = pd.DataFrame({'key1':['foo','foo','bar'],
                    'key2':['one','two','three'],
                    'lval':[1, 2, 3]})

right = pd.DataFrame({'key1':['foo','foo','bar','bar'],
                     'key2':['one','one','one','two'],
                     'rval':[4, 5, 6, 7]})

left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,three,3


In [36]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [37]:
pd.merge(left, right, on = ['key1','key2'], how = 'outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,three,3.0,NaN
4,bar,one,NaN,6.0
5,bar,two,NaN,7.0


`merge` has a `suffixes` option for specifying strings to append to overlapping names in hte left and right DataFrame objects

In [38]:
pd.merge(left, right, on = 'key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,three,3,one,6
5,bar,three,3,two,7


In [39]:
pd.merge(left, right, on = 'key1', suffixes = ('_left','_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,three,3,one,6
5,bar,three,3,two,7


*Table 8-2 merge function arguments*

---

|**Argument**|**Description**|
|:---:|:---|
|`left`|Dataframe to be merged on the left side|
|`right`|Dataframe to be merged on the right side|
|`how`|One of `inner`, `outer`, `left`, or `right`; defaults to `inner`|
|`on`|Column names to join on. Must be found in both Dataframe objects. If not specified and no other join key given, will use the intersection of the column names in `left` and `right` as the join keys.|
|`left_on`|Columns in `left` Dataframe to use as join keys|
|`right_on`|Analogous to `left_on` for `left` Dataframe|
|`left_index`|Use row index in `left` as its join key|
|`right_index`|Analogous to `left_index`|
|`sort`|Sort merged data lexicographically by join keys; `True` by default|
|`suffixes`|Tuple of string values to append to column names in case of overlap; defaults to (`_x`, `_y`)|
|`copy`|If `False`, avoid copying data into resulting data structure in some exceptional cases; by default always copies.|

### Merging on Index

In some cases, the merge key(s) in a Dataframe will be found in its index. You can pass `left_index = True` or `right_index = True` (or both) to indicate that the index should be used as the merge key

In [40]:
left1 = pd.DataFrame({'key':['a','b','a','a','b','c'],
                     'value': range(6)})
right1 = pd.DataFrame({'group_val':[3.5, 7]}, index = ['a','b'])

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [41]:
right1

,group_val
a,3.5
b,7.0


In [42]:
pd.merge(left1, right1, left_on = 'key', right_index = True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [43]:
pd.merge(left1, right1, left_on = 'key', right_index = True, how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [44]:
lefth = pd.DataFrame({'key1':['Ohio','Ohio','Ohio','Nevada','Nevada'],
                     'key2':[2000, 2001, 2002, 2001, 2000],
                     'data':np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                     index = [['Nevada','Nevada','Ohio','Ohio','Ohio','Ohio'],
                             [2001, 2000, 2002, 2001, 2001, 2001]],
                     columns = ['event1','event2'])

lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2000,4.0


In [45]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2002       4       5
       2001       6       7
       2001       8       9
       2001      10      11

In [46]:
pd.merge(lefth, righth, left_on = ['key1','key2'], right_index = True)

,key1,key2,data,event1,event2
1,Ohio,2001,1.0,6,7
1,Ohio,2001,1.0,8,9
1,Ohio,2001,1.0,10,11
2,Ohio,2002,2.0,4,5
3,Nevada,2001,3.0,0,1
4,Nevada,2000,4.0,2,3


In [47]:
pd.merge(lefth, righth, left_on = ['key1','key2'], right_index = True, how = 'outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,NaN,NaN
1,Ohio,2001,1.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
1,Ohio,2001,1.0,10.0,11.0
2,Ohio,2002,2.0,4.0,5.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2000,4.0,2.0,3.0


In [48]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index = ['a','c','e'],
                    columns = ['Ohio','Nevada'])

left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [49]:
right2 = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[13,14]],
                     index = ['b','c','d','e'],
                     columns = ['Missouri','Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [50]:
pd.merge(left2, right2, how = 'outer', left_index = True, right_index = True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [51]:
# Dataframe has a convenient join instance for merging by index. 
# It can be used to combine together many Dataframe objects having the same or similar 
# indexes but non-overlapping columns

left2.join(right2, how = 'outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [52]:
# Dataframe's join method performs a left join on the join keys, exactly perserving the 
# left frame's row index.

left1.join(right1, on = 'key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [53]:
another = pd.DataFrame([[7.,8.],[9.,10.],[11.,12.],[16.,17.]],
                      index = ['a','c','e','f'],
                      columns = ['New York','Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [54]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [55]:
left2.join([right2, another], how = 'outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating along an axis

NumPy's `concatenate` function can do with NumPyarrays

In [56]:
arr = np.arange(12).reshape((3,4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [57]:
np.concatenate([arr, arr], axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In the context of pandas objects, having labeled axes enable you to further generalize array concatenation. You have a number of additional things to think about:

* If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the shared values?
* Do the concatenated chunks of data need to be identifiable in the resulting object?
* Does the "concatenation axis" contain data that needs to be preserved?

The `concat` funcion in pandas provides a consistent way to address each of these concerns

In [58]:
s1 = pd.DataFrame([0,1], index = ['a','b'])

s2 = pd.DataFrame([2, 3, 4], index = ['c','d','e'])

s3 = pd.DataFrame([5, 6], index = ['f','g'])

pd.concat([s1, s2, s3])

,0
a,0
b,1
c,2
d,3
e,4
f,5
g,6


By default `concat` works along `axis = 0`, producing another Series. If you pass `axis =1`, the result will instead be a DataFrame

In [59]:
pd.concat([s1, s2, s3], axis = 1)

,0,0,0
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the sorted union (the '`outer`' join) of the indexes. You can instead intersect them by passing `join = 'inner'`

In [60]:
s4 = pd.concat([s1, s3])

In [61]:
s4

,0
a,0
b,1
f,5
g,6


In [62]:
pd.concat([s1, s4], axis = 1)

,0,0
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [63]:
pd.concat([s1, s4], axis = 1, join = 'inner')

,0,0
a,0,0
b,1,1


A potential issue is that the concatenated pieces are not identifiable in the result. Suppose instead you wanted to create a hierarchical index on the concatenation axis. To do this, use the `keys` argument

In [64]:
result = pd.concat([s1, s2, s3], keys = ['one','two','three'])

In [65]:
result

0
one   a  0
      b  1
two   c  2
      d  3
      e  4
three f  5
      g  6

In [66]:
result.unstack()

0                              
         a    b    c    d    e    f    g
one    0.0  1.0  NaN  NaN  NaN  NaN  NaN
two    NaN  NaN  2.0  3.0  4.0  NaN  NaN
three  NaN  NaN  NaN  NaN  NaN  5.0  6.0

In the case of combining Series along `axis = 1`, the `keys` become the DataFrame column headers:

In [67]:
pd.concat([s1, s2, s3], axis = 1, keys = ['one','two','three'])

,one,two,three
,0,0,0
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [68]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index = ['a','b','c'],
columns = ['one','two'])

In [69]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [70]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2,2), index = ['a','c'],
columns = ['three','four'])

In [71]:
df2

,three,four
a,5,6
c,7,8


In [72]:
pd.concat([df1, df2], axis = 1, keys = ['level1','level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [73]:
pd.concat({'level1':df1, 'level2':df2})

one  two  three  four
level1 a  0.0  1.0    NaN   NaN
       b  2.0  3.0    NaN   NaN
       c  4.0  5.0    NaN   NaN
level2 a  NaN  NaN    5.0   6.0
       c  NaN  NaN    7.0   8.0

In [74]:
pd.concat([df1, df2], axis = 1, keys = ['level1','level2'], names = ['upper','lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [75]:
df1 = pd.DataFrame(np.random.randn(3,4), columns = ['a','b','c','d'])

df2 = pd.DataFrame(np.random.randn(2,3), columns = ['b','d','a'])

df1

,a,b,c,d
0,0.327167,0.549356,1.144780,0.587749
1,0.668623,0.486223,-1.218172,-0.371042
2,-1.051169,0.366353,0.377968,-1.144089


In [76]:
df2

,b,d,a
0,0.990068,1.425627,-0.342786
1,-1.696290,1.443714,-1.347229


In [77]:
pd.concat([df1,df2], ignore_index=True)

,a,b,c,d
0,0.327167,0.549356,1.144780,0.587749
1,0.668623,0.486223,-1.218172,-0.371042
2,-1.051169,0.366353,0.377968,-1.144089
3,-0.342786,0.990068,NaN,1.425627
4,-1.347229,-1.696290,NaN,1.443714


### Combining data with overlap

In [78]:
a = pd.Series([np.nan, 2.5, 0.0, 3.6, 2.4, np.nan], index = ['f','e','d','c','b','a'])

b = pd.Series([0., np.nan, 2., np.nan, np.nan, 4.], index = ['a','b','c','d','e','f'])

a

f    NaN
e    2.5
d    0.0
c    3.6
b    2.4
a    NaN
dtype: float64

In [79]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    4.0
dtype: float64

In [80]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.6, 2.4, 4. ])

Series has a `combine_first` method, which performs the equivalent of this operation along with pandas's usual data alignment logic:

In [81]:
b.combine_first(a)

a    0.0
b    2.4
c    2.0
d    0.0
e    2.5
f    4.0
dtype: float64

With DataFrames, `combine_first` does the same thing column by column.

In [82]:
df1 = pd.DataFrame({'a':[1.,np.nan,4.,np.nan], 'b':[np.nan,4.,np.nan,9.], 'c':range(2, 18, 4)})

df2 = pd.DataFrame({'a':[5., 6, np.nan, 2., 9.], 'b':[np.nan, 3., 4., 4.,7.]})

df1

,a,b,c
0,1.0,NaN,2
1,NaN,4.0,6
2,4.0,NaN,10
3,NaN,9.0,14


In [83]:
df2

,a,b
0,5.0,NaN
1,6.0,3.0
2,NaN,4.0
3,2.0,4.0
4,9.0,7.0


In [84]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,6.0,4.0,6.0
2,4.0,4.0,10.0
3,2.0,9.0,14.0
4,9.0,7.0,NaN


## Reshaping and Pivoting

There are number of basic operations for rearranging tabular data. These are alternatingly referred to as *reshape* or *pivot* operations.

### Reshaping with hierarchical indexing

Hierarchical indexing provides a consistent way to rearrange data in a DataFrame. There are two primary actions:

`stack`
    This "rotates" or pivots from thecolumns in the data to the rows.

`unstack`
    This pivots from the rows into the columns

In [85]:
data = pd.DataFrame(np.arange(6).reshape((2,3)),
index = pd.Index(['Ohio','Colorado'], name = 'state'),
columns = pd.Index(['one','two','three'], name = 'number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [86]:
result = data.stack()

result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [87]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [88]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [89]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [90]:
# Unstacking might introduce missing data if all of the values in the level are not found in each of the subgroups:
s1 = pd.Series([0,1,2,3], index = ['a','b','c','d'])

s2 = pd.Series([4,5,6], index = ['c','d','e'])

s1

a    0
b    1
c    2
d    3
dtype: int64

In [91]:
s2

c    4
d    5
e    6
dtype: int64

In [92]:
data2 = pd.concat([s1,s2], keys = ['one','two'])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [93]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [94]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [95]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [96]:
data2.unstack().stack(dropna = False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [97]:
df = pd.DataFrame({'left': result, 'right': result + 5}, columns = pd.Index(['left','right'], name = 'side'))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [98]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [99]:
df.unstack('number')

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [100]:
# When calling stack, we can indicate the name of the axis to stack:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Long" to "Wide" format

A common way to store multiple time series in databases and CSV is in so-called *long* or *stacked* format.

In [101]:
data = pd.read_csv('datasets/macrodata.csv')

data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [102]:
periods = pd.PeriodIndex(year = data.year, quarter = data.quarter, name = 'date')

columns = pd.Index(['realgdp','infl','unemp'],name = 'item')

data = data.reindex(columns = columns)

data.index = periods.to_timestamp('D', 'end')

ldata = data.stack().reset_index().rename(columns = {0: 'value'})

ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


This is the so-called *long* format for multiple time series, or other observational data with tow or more keys. Each row in the table represents a single observation.

Data is frequently stored this way in relational databases like MySQL, as a fixed schema allows the number of distinct values in the `item` column to change as data is added to the table.

In some cases, the data may be more difficult to work with this format; you might prefer to have a DataFrame containing one column per distinct `item` value indexed by timestamps in the `data` column.

In [103]:
pivoted = ldata.pivot('date','item','value')

pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


The first two values passed are the columns to be used repectively as the row and column index, then finally an optional value column to fill the DataFrame

In [104]:
ldata['value2'] = np.random.randn(len(ldata))

ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.566442
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.212830
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.545476
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.052373
4,1959-06-30 23:59:59.999999999,infl,2.340,0.789675
5,1959-06-30 23:59:59.999999999,unemp,5.100,-1.250773
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-1.355282
7,1959-09-30 23:59:59.999999999,infl,2.740,0.394912
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.779643
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.736075


In [105]:
pivoted = ldata.pivot('date','item')

pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.212830 -1.566442   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.789675 -0.052373   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.394912 -1.355282   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.798838 -0.736075   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.407313  1.508401   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.545476  
1959-06-30 23:59:59.999999999 -1.250773  
1959-09-30 23:59:59.999999999  0.779643  
1959-12-31 23:59:59.999999999  0.522441  
1960-03-31 23:59:59.999999999  2.337937

In [106]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [107]:
# pivot is equivalent to creating a hierarchical index using set_index folowed by a call to unstack

unstacked = ldata.set_index(['date','item']).unstack('item')

unstacked[:8]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.212830 -1.566442   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.789675 -0.052373   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3  0.394912 -1.355282   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.798838 -0.736075   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -0.407313  1.508401   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2  0.719097 -0.169828   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.514430  0.960690   
1960-12-31 23:59:59.999999999  1.21  2802.616   6.3 -0.387225 -1.188769   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.545476  
1959-06-30 23:59:59.999999999 -1.250773  
1959-09-30 23:59:59.999999999  0.779643  
1959-12-31 23:59:59.999999999  0.522441  
1960-03-31 23:59:59.999999999  2.337937  
1960-06-30 23:59:59.999999999  0.096867  
1960-09-30 23:59:59.999999999 -1.452672  
1960-12-31 23:59:59.999999999  0.084791

### Pivoting "wide" to "long" format

An inverse operation to `pivot` for DataFrames is `pandas.melt`. Rather than transforming one column into many in a new DataFrame, it merges multiple columns into one, producing a DataFrame that is longer than the input

In [108]:
df = pd.DataFrame({'key':['foo','bar','baz'], 
'A':[1,2,3],
'B':[4,5,6],
'C':[7,8,9]})

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


The `key` column may be a group indicator, and the other columns are data values. When using `pandas.melt`, we must indicate which columns are group indicators.

In [109]:
melted = pd.melt(df,['key'])

melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [110]:
# Using pivot, we can reshape back to the original layout.

reshaped = melted.pivot('key','variable','value')

reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [111]:
# Since the result of `pivot creates an index from the column used as the row labels, we may want to use reset_index to move the data back into a column

reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [112]:
# Specify a subset of columns to use as value columns.

pd.melt(df, id_vars = ['key'], value_vars = ['A','B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [113]:
# Can be used without any group identifiers.

pd.melt(df, value_vars = ['A','B','C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [114]:
pd.melt(df, value_vars = ['key','A','B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


## Data Aggregation and Group Operations

Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often a critical component of a data analysis workflow. After loading, merging, and preparing a dataset, you may need to compute group statistics or possibly *pivot tables* for reporting or visualization purposes. pandas provides a flexible `groupby` interface, enabling you to slice, dice, and summarize datasets in a natural way.

###GroupBy mechanics

The splitting is performed on a particular axis of an object. 
![alt text here](images/groupby.jpg)
Each grouping key can take many forms, and the keys do not have to be all the same type:
* A list or array of values that is the same length as the axis being grouped.
* A value indicating a column name in a DataFrame.
* A dict or Series giving a correspondence between the values on the axis being grouped and the group names.
A function to be invoked on the axis index or the individual labels in the index.

Note that the latter three methods are shortcuts for producing an array of values to be used to split up the object.




In [115]:
df = pd.DataFrame({'key1':['a','b','a','b','a'],
'key2':['one','two','one','two','one'],
'data1': np.random.randn(5),
'data2': np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-0.248845,1.253614
1,b,two,-0.305055,0.862469
2,a,one,0.592339,0.843221
3,b,two,-2.006196,-0.249327
4,a,one,-1.421291,0.486239


In [116]:
grouped = df['data1'].groupby(df['key1'])

grouped

This `grouped` variable is now a *GroupBy* object. It has not actually computed anything yet except for some intermediate data about the group key `df['key1']`. The idea is that this object has all of the information needed to then apply some operation to each of the groups.

In [117]:
grouped.mean()

key1
a   -0.359266
b   -1.155625
Name: data1, dtype: float64

In [118]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [119]:
means

key1  key2
a     one    -0.359266
b     two    -1.155625
Name: data1, dtype: float64

In [120]:
means.unstack()

key2,one,two
key1,,
a,-0.359266,NaN
b,NaN,-1.155625


In [121]:
# The group keys are all Series, though they could be any arrays of the right length:

states = np.array(['Ohio','California','California','Ohio','Ohio'])

years = np.array([2005, 2007, 2005, 2007, 2007])

df['data1'].groupby([states, years]).mean()

California  2005    0.592339
            2007   -0.305055
Ohio        2005   -0.248845
            2007   -1.713744
Name: data1, dtype: float64

In [122]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.359266,0.861025
b,-1.155625,0.306571


In [123]:
df.groupby(['key1','key2']).mean()

,,data1,data2
key1,key2,,
a,one,-0.359266,0.861025
b,two,-1.155625,0.306571


In the first case `df.gropby('key1').mean()` that there is no `key2` column in the result. Because `df['key2']` is not numeric data, it is said to be a *nuisance* column, which is therefore excluded from the result.

In [124]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     3
b     two     2
dtype: int64

### Iterating over groups

In [125]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.248845  1.253614
2    a  one  0.592339  0.843221
4    a  one -1.421291  0.486239
b
  key1 key2     data1     data2
1    b  two -0.305055  0.862469
3    b  two -2.006196 -0.249327


In [126]:
for (k1, k2), group in df.groupby(['key1','key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.248845  1.253614
2    a  one  0.592339  0.843221
4    a  one -1.421291  0.486239
('b', 'two')
  key1 key2     data1     data2
1    b  two -0.305055  0.862469
3    b  two -2.006196 -0.249327


In [127]:
pieces = dict(list(df.groupby('key1')))

In [128]:
pieces['b']

,key1,key2,data1,data2
1,b,two,-0.305055,0.862469
3,b,two,-2.006196,-0.249327


In [129]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [130]:
for dtype, group in grouped:
    print(dtype)
    print(group)

a
0   -0.248845
2    0.592339
4   -1.421291
Name: data1, dtype: float64
b
1   -0.305055
3   -2.006196
Name: data1, dtype: float64


### Selecting a column or subset of columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation.

In [131]:
df.groupby('key1')['data1']

In [132]:
df.groupby('key1')[['data2']]

In [133]:
df['data1'].groupby(df['key1'])

In [134]:
df['data2'].groupby(df['key1'])

In [135]:
df.groupby(['key1','key2'])[['data2']].mean()

,,data2
key1,key2,
a,one,0.861025
b,two,0.306571


In [136]:
# The object returned by this indexing operation is a grouped DataFrame if a list or array is passed or a grouped Series if only a single column name is passed as a scalar:

s_grouped = df.groupby(['key1','key2'])['data2']

s_grouped

In [137]:
s_grouped.mean()

key1  key2
a     one     0.861025
b     two     0.306571
Name: data2, dtype: float64

### Grouping with dicts and series

Grouping information may exist in a form other than an array. 

In [138]:
people = pd.DataFrame(np.random.randn(5, 5),
columns = ['a','b','c','d','e'],
index = ['Joe','Steve','Alice','Ben','Pitt'])

people.iloc[2:3, [1,2]] = np.nan  # Add a few NA values

people

,a,b,c,d,e
Joe,-0.642451,-0.098607,-0.807573,0.833457,2.253346
Steve,-1.101707,0.341095,0.418109,0.346546,-0.134675
Alice,-0.262594,NaN,NaN,1.298169,-0.353595
Ben,0.162826,0.684781,-0.117119,-0.386679,0.810201
Pitt,-0.229247,-0.242969,-1.313454,-1.637738,0.317140


In [139]:
# Suppose having a group correspondence for the columns and want to sum together the columns by group:

mapping = {'a': 'red', 'b':'red', 'c':'blue',
'd':'blue','e':'red','f': 'orange'}

In [140]:
# Construct an array from this dict to pass to grouby, but instead we can just pass the dict.a
by_column = people.groupby(mapping, axis = 1)

by_column.sum()

,blue,red
Joe,0.025885,1.512287
Steve,0.764656,-0.895287
Alice,1.298169,-0.616189
Ben,-0.503798,1.657807
Pitt,-2.951192,-0.155077


In [141]:
# The same functionality holds for Series, which can be viewed as a fixed-size mapping:

map_series = pd.Series(mapping)

map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [142]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Alice,1,2
Ben,2,3
Pitt,2,3


### Grouping with Functions

Using Python functions is a more generic way of defining a group mapping compared with a dict or Series. Any function passed as a group key will be called once per index value, with the return values being used as the group names.

In [143]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.479625,0.586173,-0.924692,0.446778,3.063547
4,-0.229247,-0.242969,-1.313454,-1.637738,0.317140
5,-1.364300,0.341095,0.418109,1.644715,-0.488270


In [144]:
# Mixing functions with arrays, dicts, or Series is not a problem as everything gets converted to arrays internally:

key_list = ['one','one','one','two','two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.642451 -0.098607 -0.807573  0.833457  2.253346
  two  0.162826  0.684781 -0.117119 -0.386679  0.810201
4 two -0.229247 -0.242969 -1.313454 -1.637738  0.317140
5 one -1.101707  0.341095  0.418109  0.346546 -0.353595

### Grouping by index levels

A final convenience for hierarchically indexed datasets is the ability to aggregate using one of the levels of an axis index.

In [145]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
[1, 3, 4, 2, 3]],
names = ['cty','tenor'])

In [146]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns = columns)

hier_df

cty          US                            JP          
tenor         1         3         4         2         3
0     -0.110015  0.296562 -0.058255  1.839921 -0.147149
1      1.302114  0.893593  1.858334  1.465343  0.305426
2      2.806917 -0.872425  0.066045  1.057759 -2.628111
3      1.856242 -0.873278  0.791874 -0.588483 -1.056665

In [147]:
# To group by level, pass the level number or name using the level keyword

hier_df.groupby(level = 'cty', axis = 1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

Aggregation refer to any data transformation that produces scalar values from arrays. The preceding examples have used several of them, including `mean`, `count`, `min`, and `min`. Many common Aggregations have optimized implementations.

---

*Table 10-1. Optimized groupby methods*

|**Function name**|**Description**|
|:---:|:---|
|`count`|Number of non-NA values in the group|
|`sum`|Sum of non-NA values|
|`mean`|Mean of non-NA values|
|`median`|Arithmetic median of non-NA values|
|`std, var`|Unbiased (n-1 deniminator) standard deviation and variance|
|`min, max`|Minimum and maximum of non-NA values|
|`prod`|Product of non-NA values|
|`first, last`|First and last non-NA values|

---

You can use Aggregations of your own devising and additionally call any method that is also defined on the grouped object.

![alt text here](images/agg.jpg)



In [148]:
df

,key1,key2,data1,data2
0,a,one,-0.248845,1.253614
1,b,two,-0.305055,0.862469
2,a,one,0.592339,0.843221
3,b,two,-2.006196,-0.249327
4,a,one,-1.421291,0.486239


In [149]:
grouped = df.groupby('key1')

In [150]:
grouped

In [151]:
grouped['data1'].quantile(0.9)

key1
a    0.424102
b   -0.475169
Name: data1, dtype: float64

In [152]:
# To use your own aggregation functions, pass any function that aggregates an array to the aggregate or agg method:

def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.013630,0.767375
b,1.701141,1.111796


In [153]:
# Some methods like describe also work, even though they are not aggregations:

grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.359266  1.011346 -1.421291 -0.835068 -0.248845  0.171747   
b      2.0 -1.155625  1.202889 -2.006196 -1.580911 -1.155625 -0.730340   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.592339   3.0  0.861025  0.383997  0.486239  0.664730  0.843221   
b    -0.305055   2.0  0.306571  0.786159 -0.249327  0.028622  0.306571   

                          
           75%       max  
key1                      
a     1.048417  1.253614  
b     0.584520  0.862469

### Column-wise and multiple function application

After loading it with read_csv, we add a tipping percentage column tip_pct:

In [154]:
tips = pd.read_csv('datasets/tips.csv')

In [155]:
tips['tips_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tips_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


Aggreating a Series or all of the columns of a DataFrame is a matter of using `aggregate` with the desired function or calling a method like `mean` or `std`. 

In [156]:
grouped = tips.groupby(['day','smoker'])

In [157]:
grouped_pct = grouped['tips_pct']

grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tips_pct, dtype: float64

In [158]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

Here we passed a list of aggregation functions to `agg` to evaluate independently on the data groups.

You don't need to accept the names that GroupBy gives to the columns; notably, `lambda` functions have the name `<lambda>`, which makes them hard to identify. If you pass a list of (`name, function`) tuples, the first element of each tuple will be used as the DataFrame column names.

In [159]:
grouped_pct.agg([('foo','mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [160]:
functions = ['count','mean','max']

results = grouped['tips_pct','total_bill'].agg(functions)

results

tips_pct                     total_bill                  
               count      mean       max      count       mean    max
day  smoker                                                          
Fri  No            4  0.151650  0.187735          4  18.420000  22.75
     Yes          15  0.174783  0.263480         15  16.813333  40.17
Sat  No           45  0.158048  0.291990         45  19.661778  48.33
     Yes          42  0.147906  0.325733         42  21.276667  50.81
Sun  No           57  0.160113  0.252672         57  20.506667  48.17
     Yes          19  0.187250  0.710345         19  24.120000  45.35
Thur No           45  0.160298  0.266312         45  17.113111  41.19
     Yes          17  0.163863  0.241255         17  19.190588  43.11

In [161]:
results['tips_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [162]:
# A list of tuples with custom names can be passed:

ftuples = [('Durchschnitt','mean'), ('Abweichung', np.var)]

grouped['tips_pct', 'total_bill'].agg(ftuples)

tips_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [163]:
# Suppose you wanted to apply potentially different functions to one or more of the columns. Pass a 
# dict to `agg` that contains a mapping of column names to any of the function specifications listed
# so far:

grouped.agg({'tip':np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [164]:
grouped.agg({'tips_pct':['min','max','mean','std'], 'size':'sum'})

tips_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning aggregated data without row indexes

In all of the examples up until now, the aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations. You can disable this behavior in most cases by passing `as_index = False`.

In [165]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tips_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## Apply: General split-apply-combine

The most general-purpose GroupBy method is `apply`, which is the subject of the rest of this section.
![alt text here](images/apply.jpg)

In [166]:
def top(df, n = 5, column = 'tips_pct'):
    return df.sort_values(by = column)[-n:]

top(tips, n = 6)

,total_bill,tip,sex,smoker,day,time,size,tips_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [167]:
# If we group by smoker, say, and call apply with this function.a
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size  tips_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [168]:
tips.groupby(['smoker','day']).apply(top, n = 1, column = 'total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                 tips_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [169]:
result = tips.groupby('smoker')['tips_pct'].describe()

In [170]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [171]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [172]:
# When you invoke a method like describe, it is actually just a shortcut for:

f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size  tips_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

### Suppressing the group keys

The resulting object has a hierarchical index formed from the group keys along with the indexes of each piece of the original object. You can disable this by passing `group_keys=False` to `groupby`

In [173]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tips_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and bucket analysis



In [174]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)

quartiles[:10]

0      (0.214, 1.717]
1     (-1.289, 0.214]
2    (-2.799, -1.289]
3     (-1.289, 0.214]
4     (-1.289, 0.214]
5     (-1.289, 0.214]
6     (-1.289, 0.214]
7      (1.717, 3.221]
8      (0.214, 1.717]
9      (0.214, 1.717]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.799, -1.289] < (-1.289, 0.214] < (0.214, 1.717] < (1.717, 3.221]]

In [175]:
# The categorical object returned by cut can be passed directly to groupby.

def get_stats(group):
    return {'min':group.min(), 'max':group.max(), 'count':group.count(), 'mean':group.mean()}

grouped = frame.data2.groupby(quartiles)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-2.799, -1.289]",-2.722241,1.969175,105.0,-0.219115
"(-1.289, 0.214]",-2.947469,3.055888,481.0,0.056012
"(0.214, 1.717]",-3.244358,4.017828,372.0,0.000260
"(1.717, 3.221]",-1.624765,1.609747,42.0,-0.004773


In [176]:
# These were equal-length buckets; to comupte equal-size buckets based on sample quantiles, use qcut.
# Return quantile numbers

grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)

grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.722241,1.969175,100.0,-0.218817
1,-2.102919,3.055888,100.0,-0.076556
2,-2.432829,2.234884,100.0,0.016535
3,-2.246667,2.010452,100.0,0.028383
4,-2.947469,2.857545,100.0,0.120175
5,-1.607793,2.420269,100.0,0.179316
6,-3.079945,2.028754,100.0,-0.089465
7,-2.565098,4.017828,100.0,-0.061335
8,-3.244358,2.367541,100.0,0.170705


### Example: filling missing values with group-specific values

When cleaning up missing data, in some cases you will replace data observations using `dropna`, but in others you may want to impute (fill in) the null (NA) values using a fixed value or some value derived from the data.  `fillna` is the right tool to use.

In [177]:
s = pd.Series(np.random.randn(6))

s[::2] = np.nan

s

0         NaN
1   -0.009925
2         NaN
3   -1.920840
4         NaN
5    2.040251
dtype: float64

In [178]:
s.fillna(s.mean())

0    0.036495
1   -0.009925
2    0.036495
3   -1.920840
4    0.036495
5    2.040251
dtype: float64

Suppose you need the fill value to vary by group. One way to do this is to group the data and use `apply` with a function that calls `fillna` on each data chunk.

In [179]:
states = ['Ohio','NY','Vermont','Florida','Oregon','Nevada','CA','Idaho']

group_key = ['East'] * 4 + ['West'] * 4

data = pd.Series(np.random.randn(8), index = states)

data

Ohio       1.373964
NY        -0.658468
Vermont    0.908406
Florida   -1.067885
Oregon    -0.654349
Nevada    -0.885859
CA         0.751873
Idaho     -0.812901
dtype: float64

In [180]:
data[['Vermont','Nevada','Idaho']] = np.nan
data

Ohio       1.373964
NY        -0.658468
Vermont         NaN
Florida   -1.067885
Oregon    -0.654349
Nevada          NaN
CA         0.751873
Idaho           NaN
dtype: float64

In [181]:
data.groupby(group_key).mean()

East   -0.117463
West    0.048762
dtype: float64

In [182]:
# Fill the NA values using the group means

fill_mean = lambda g: g.fillna(g.mean())

data.groupby(group_key).apply(fill_mean)

Ohio       1.373964
NY        -0.658468
Vermont   -0.117463
Florida   -1.067885
Oregon    -0.654349
Nevada     0.048762
CA         0.751873
Idaho      0.048762
dtype: float64

In [183]:
fill_values = {'East': 0.5, 'West': -1}

fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

Ohio       1.373964
NY        -0.658468
Vermont    0.500000
Florida   -1.067885
Oregon    -0.654349
Nevada    -1.000000
CA         0.751873
Idaho     -1.000000
dtype: float64

### Example: random sampling and permutation

Suppose you wanted to draw a random sample (with or without replacement) from a large dataset for Monte Carlo simulation purposes. There are a number of ways to perform the "draws"

In [184]:
# Heart, Spades, Clubs, Diamonds

suits = ['H','S','C','D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J','K','Q']
cards = []

for suit in ['H','S','C','D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index = cards)

In [185]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [186]:
def draw(deck, n = 5):
    return deck.sample(n)

draw(deck)

9S     9
KH    10
2C     2
3S     3
3C     3
dtype: int64

In [187]:
# Suppose you wanted two random cards from each suit.

get_suit = lambda card: card[-1]  # last letter is suit

deck.groupby(get_suit).apply(draw, n = 2)

C  QC    10
   9C     9
D  8D     8
   5D     5
H  9H     9
   4H     4
S  JS    10
   8S     8
dtype: int64

In [188]:
# Alternatively, we could write:

deck.groupby(get_suit, group_keys = False).apply(draw, n = 2)

4C      4
QC     10
6D      6
2D      2
2H      2
10H    10
2S      2
6S      6
dtype: int64

## Pivot Tables and Cross-Tabulation

A *pivot table* is a data summarization tool frequently found in spreadsheet programs and other data analysis software. It aggregates a table of data by one or more keys, arranging the data in a rectanble with some of the group keys along the rows and some along the columns.

In [189]:
tips.pivot_table(index = ['day','smoker'])

size       tip  tips_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [190]:
# This could have been produced with groupby directly.
# Suppose we want to aggregate only tip_pct and size, and additionally group by time.

tips.pivot_table(['tips_pct', 'size'], index = ['time','day'], columns = 'smoker')

size            tips_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [191]:
# We could augment this table to include partial totals by passing margins = True. 

tips.pivot_table(['tips_pct','size'], index = ['time','day'], columns = 'smoker', margins=True)

size                      tips_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [192]:
# To use a different aggregation function, pass it to aggfunc.

tips.pivot_table('tips_pct', index = ['time','smoker'], columns = 'day', aggfunc = len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [193]:
# If some combinations are empty, you may wish to pass a fill_value. 

tips.pivot_table('tips_pct', index = ['time','size','smoker'], columns = 'day', aggfunc = 'mean', fill_value = 0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab

A cross-tabulation (or *crosstab* for short) is a special case of a pivot table that computers group frequencies

In [194]:
df = pd.read_csv('datasets/facebookac.csv')
df.head()

,Status,No Friend,education,about me,family,gender,relationship,phototag*,photopost*,video,...,music,film,series,book,game,restaurant,like,group,note,post shared/post posted rate
0,real,170,university,yes,yes,male,complicate,29,59,8,...,1,3,6,0,6,2,101,2,yes,0.1
1,real,353,university,yes,yes,male,alone,1,13,0,...,11,6,10,1,1,6,550,19,no,0.1
2,real,517,university,no,yes,male,alone,112,236,3,...,0,8,3,1,5,17,318,23,yes,0.5
3,real,460,university,no,yes,male,alone,74,142,3,...,16,14,17,6,19,0,900,32,yes,0.3
4,real,240,university,no,yes,female,complicate,23,13,1,...,0,0,0,0,0,0,15,2,no,0.1


In [195]:
df_n = df.dropna()
df.columns

Index(['Status', 'No Friend', 'education', 'about me', 'family', 'gender',
       'relationship', 'phototag*', 'photopost*', 'video', 'checkin', 'sport',
       'player', 'music', 'film', 'series', 'book', 'game', 'restaurant',
       'like', 'group', 'note', 'post shared/post posted rate'],
      dtype='object')

In [196]:
df = df.replace(' ', 'not available')

In [197]:
df.head()

,Status,No Friend,education,about me,family,gender,relationship,phototag*,photopost*,video,...,music,film,series,book,game,restaurant,like,group,note,post shared/post posted rate
0,real,170,university,yes,yes,male,complicate,29,59,8,...,1,3,6,0,6,2,101,2,yes,0.1
1,real,353,university,yes,yes,male,alone,1,13,0,...,11,6,10,1,1,6,550,19,no,0.1
2,real,517,university,no,yes,male,alone,112,236,3,...,0,8,3,1,5,17,318,23,yes,0.5
3,real,460,university,no,yes,male,alone,74,142,3,...,16,14,17,6,19,0,900,32,yes,0.3
4,real,240,university,no,yes,female,complicate,23,13,1,...,0,0,0,0,0,0,15,2,no,0.1


In [198]:
pd.crosstab(df['relationship'], df['Status'])

Status,fake,real
relationship,,
alone,418,353
complicate,14,64
married,0,15
not available,3,22


In [199]:
pd.crosstab(df['relationship'], df['Status'], margins=True, margins_name='Total')

Status,fake,real,Total
relationship,,,
alone,418,353,771
complicate,14,64,78
married,0,15,15
not available,3,22,25
Total,435,454,889


In [200]:
pd.crosstab(df['relationship'], df['Status'], normalize=True)

Status,fake,real
relationship,,
alone,0.470191,0.397075
complicate,0.015748,0.071991
married,0.000000,0.016873
not available,0.003375,0.024747


In [201]:
pd.crosstab([df['relationship'], df['Status']], df['gender'], normalize=1)

gender                  female      male
relationship  Status                    
alone         fake    0.587473  0.342723
              real    0.308855  0.492958
complicate    fake    0.017279  0.014085
              real    0.038877  0.107981
married       real    0.012959  0.021127
not available fake    0.000000  0.007042
              real    0.034557  0.014085

## Advanced pandas

### Categorical data

#### Background and motivation

Frequently, a column in a table may contain repeated instances of a smaller set of distinct values.

In [202]:
values = pd.Series(['apple','orange','apple','apple'] * 2)

values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [203]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [204]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In data warehousing, a best practice is to use so-called *dimension tables* containing the distinct values and storing the primary observations as integer keys referencing the dimension table

In [205]:
values = pd.Series([0, 1, 0, 0] * 2)

dim = pd.Series(['apple','orange'])

values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [206]:
dim

0     apple
1    orange
dtype: object

In [207]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

This representation as integers is called the *categorical* or *dictionary-encoded* representation. The array of distinct values can be called the *categories, dictionary,* or *levels* of the data. The integer values that reference the categorries are called the *category codes* or simply *codes*.

The categorical representation can yield significant performance improvements when you are doing analytics. You can also perform transformations on the categories while leaving the codes unmodified.

#### Categorical type in pandas

pandas has a special `Categorical* type for holding data that uses the integer-based categorical representation or *encoding*.

In [208]:
fruits = ['apple','orange','apple','apple'] * 2

N = len(fruits)

df = pd.DataFrame({'fruit': fruits, 'basket_id':np.arange(N), 'count':np.random.randint(3, 15, size=N), 'weight':np.random.uniform(0, 4, size=N)}, columns = ['basket_id','fruit','count','weight'])

df

,basket_id,fruit,count,weight
0,0,apple,9,0.402473
1,1,orange,12,1.943740
2,2,apple,6,0.154543
3,3,apple,11,3.413788
4,4,apple,8,2.233013
5,5,orange,7,0.351828
6,6,apple,7,0.663507
7,7,apple,9,2.865975


In [209]:
# df['fruit'] is an array of Python string objects. We can convert it to categorical by calling:

fruit_cat = df['fruit'].astype('category')

fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [210]:
# The values for fruit_cat are not a NumPy array, but an instance of pandas.Categorical:

c = fruit_cat.values

In [211]:
type(c)

pandas.core.arrays.categorical.Categorical

In [212]:
# Convert a Dataframe column to categorical by assigning the converted result:

df['fruit'] = df['fruit'].astype('category')
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [213]:
# Create pandas.Categorical directly from other types of Python sequences:

my_categories = pd.Categorical(['foo','bar','baz','foo','bar'])

my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [214]:
# If you have obtained categorical encoded data from another source, you can use the alternative from_codes constructor:

categories = ['foo','bar','baz']

codes = [0 ,1, 2, 0, 0, 1]

my_cats_2 = pd.Categorical.from_codes(codes, categories)

my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [215]:
# Unless explicitly specified, categorical conversions assume no specific ordering of the categories. So the categories array may be in a different order depending on the ordering of the input data. When using from_codes or any of the other constructors, you can indicate that the categories have a meaningful ordering:

ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)

ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [216]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

#### Computations with categoricals

Using `Categorical` in pandas compared with the non-encoded version generally behaves thesame way. Some parts of pandas, like `groupby` function, perform better when working with categoricals. There are also some fnctions that can utilize the `ordered` flag.

In [217]:
np.random.seed(12345)

draws = np.random.randn(1000)

draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [218]:
bins = pd.qcut(draws, 4)

bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [219]:
bins = pd.qcut(draws, 4, labels = ['Q1','Q2','Q3','Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [220]:
# The labeled bins categorical does not contain information about the bin edges in the data, so we can use groupby
# to extract some summary statistics:

bins = pd.Series(bins, name = 'quartile')

results = (pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())

results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [221]:
# The 'quartile' column in the result retains the original categorical information, including ordering, from bins:

results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

### Better performance with categoricals

If you do a lot of analytics on a particular dataset, converting to categorical can yield substantial overall performance gains. A categorical version of a Dataframe column will often use significantly less memory, too.

In [222]:
N = 10000000

draws = pd.Series(np.random.randn(N))

labels = pd.Series(['foo','bar','baz','qux'] * (N // 4))

In [223]:
categories = labels.astype('category') # Convert labels to categorical

In [224]:
labels.memory_usage()

80000128

In [225]:
categories.memory_usage()

10000320

In [226]:
# Note that labels uses significantly more memory than categories
# The conversion to category is not free, but it is one-time cost

%time _ =labels.astype('category')

Wall time: 576 ms


#### Categorical Methods

Series containing Categorical data have several special methods similar to the `Series.str` specialized string methods. This also provides convenient access to the categories and codes.

In [227]:
s = pd.Series(['a','b','c','d'] * 2)

cat_s = s.astype('category')

cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [228]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [229]:
# Suppose that we know the actual set of categories for this data extends beyond the four values observed in data. 
# We can use the set_categories method to change them.
actual_categories = ['a','b','c','d','e']

cat_s2 = cat_s.cat.set_categories(actual_categories)

cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [230]:
# While it appears that the data is unchanged, the new categories will be reflected in operations that use them.

cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [231]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

In large datasets, catgoricals are often used as a convenient tool for memory savings and better performance. After you filter a large DataFrame or Series, many of the categories may not appera in the data.

In [232]:
cat_s3 = cat_s[cat_s.isin(['a','b'])]

cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [233]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

### Creating dummy variables for modeling

when you're using statistics or machine learning tools, you will often transform categorical data into *dummy variables*, aka *one-hot* encoding. This involves creating a DataFrame with a column for each distinct category; these columns contain 1s for occurrences of a given category and 0 otherwise.

In [234]:
cat_s = pd.Series(['a','b','c','d'] * 2, dtype = 'category')

In [235]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## Advanced GroupBy use

### Group transforms and "unwrapped" groupbys

* It can produce a scalar value to be broadcast to the shape of the group.
* It can produce an object of the same shape as the input group.
* It must not mutate its input

In [236]:
df = pd.DataFrame({'key':['a','b','c'] * 4, 'value':np.arange(12.)})

In [237]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [238]:
g = df.groupby('key').value

In [239]:
g

In [240]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

In [241]:
# Suppose instead we wanted to produce a Series of the same shape as df['value'] but with values replaced by the 
# average grouped by 'key'. We can pass the function lambda x: x.mean() to transform.

g.transform(lambda x: x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [242]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [243]:
g.transform(lambda x:x * 2)

0      0.0
1      2.0
2      4.0
3      6.0
4      8.0
5     10.0
6     12.0
7     14.0
8     16.0
9     18.0
10    20.0
11    22.0
Name: value, dtype: float64

In [244]:
g.transform(lambda x:x.rank(ascending = False))

0     4.0
1     4.0
2     4.0
3     3.0
4     3.0
5     3.0
6     2.0
7     2.0
8     2.0
9     1.0
10    1.0
11    1.0
Name: value, dtype: float64

In [245]:
def normalize(x):
    return (x - x.mean())/x.std()

In [246]:
g.transform(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [247]:
g.apply(normalize)

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

In [248]:
# Build in aggregae functions like 'mean' or 'sum' are often much faster than a general apply function 
# These also have a "fast past" when used with transform.
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [249]:
normalized = (df['value'] - g.transform('mean')) / g.transform('std')

In [250]:
normalized

0    -1.161895
1    -1.161895
2    -1.161895
3    -0.387298
4    -0.387298
5    -0.387298
6     0.387298
7     0.387298
8     0.387298
9     1.161895
10    1.161895
11    1.161895
Name: value, dtype: float64

#### Groped time resampling

For time series data, the `resample` method is semanitically a group operation based on a time intervalization.

In [252]:
N = 15

times = pd.date_range('2017-5-20 00:00', freq = '1min', periods = N)

df = pd.DataFrame({'time': times, 'value': np.arange(N)})

In [253]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


In [255]:
# Index by 'time' and then resample.
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [257]:
# Suppose that a DataFrame contains multiple time series, marked by an additional group key column.
df2 = pd.DataFrame({'time':times.repeat(3), 'key': np.tile(['a','b','c'], N), 'value':np.arange(N * 3.)})

df2[:7]

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [265]:
# To do the same resampling for each value of 'key', we introduce the pandas.TimeGrouper object

time_key = pd.Grouper('5min')
time_key

Grouper(key='5min', axis=0, sort=False)

In [266]:
resampled = (df2.set_index('time').groupby(['key', time_key]).sum())

KeyError: 'The grouper name 5min is not found'